In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pickle.load(open("dataset_level2.pkl",'rb'))

In [3]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,New Wanderers Stadium
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,New Wanderers Stadium
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,New Wanderers Stadium
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,New Wanderers Stadium
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,New Wanderers Stadium


In [4]:
df.shape

(55046, 8)

**Feature Extraction**

* order 8 col
* batting_team, bowling_team, current_score, wicket_left,crr(current run rate), city, ball_left, last_five

In [5]:
df.isna().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                7735
venue                  0
dtype: int64

In [6]:
df[df['city'].isnull()][['city','venue']].head()

,city,venue
0,NaN,Melbourne Cricket Ground
1,NaN,Melbourne Cricket Ground
2,NaN,Melbourne Cricket Ground
3,NaN,Melbourne Cricket Ground
4,NaN,Melbourne Cricket Ground


In [7]:
df[df['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        2721
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1328
Sydney Cricket Ground                       749
Sharjah Cricket Stadium                     249
Harare Sports Club                          248
Adelaide Oval                               246
Sylhet International Cricket Stadium        128
Name: venue, dtype: int64

* In this ground in which city location is null and we want to fill on the bases of ground loacation 
* most of the city name is ground first word

In [8]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

* np.where(df['city'].isnull() ---->True --- df['venue'].str.split().apply(lambda x:x[0])

* np.where(df['city'].isnull() ----> False --- df['city] remain as it

In [9]:
cities

array(['Johannesburg', 'Johannesburg', 'Johannesburg', ..., 'Colombo',
       'Colombo', 'Colombo'], dtype=object)

In [10]:
df['city'] = cities

In [11]:
df['city'].isna().value_counts()

False    55046
Name: city, dtype: int64

* it show that there is no null value in city

In [12]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [13]:
df.drop(columns=['venue'], inplace = True)

In [14]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg


* In most of the city - match played  single time --> so require the city which have occurs atleast 5 match in that city-- 5 match*120 =600 single inning(data)

In [15]:
df['city'].value_counts().tail()

Dharamsala       122
Potchefstroom    122
St Vincent       122
Ahmedabad        121
Cuttack          114
Name: city, dtype: int64

In [16]:
df['city'].value_counts()[df['city'].value_counts()>600]

Colombo            3707
Mirpur             3174
Dubai              2721
Johannesburg       2461
London             2104
Pallekele          2066
Auckland           2033
Barbados           1737
Cape Town          1618
Sydney             1579
St Lucia           1487
Melbourne          1449
Lauderhill         1238
Wellington         1234
Durban             1161
Southampton         987
Hamilton            972
Centurion           943
Mumbai              873
Nottingham          866
Kolkata             844
Chittagong          745
Delhi               739
Abu Dhabi           734
Mount Maunganui     693
Manchester          673
Chandigarh          627
Lahore              624
St Kitts            618
Christchurch        614
Trinidad            611
Nagpur              610
Cardiff             607
Name: city, dtype: int64

In [17]:
df['city'].value_counts()[df['city'].value_counts()>600].shape

(33,)

* so minimun 33 city which have match occurs more than 5 times

In [18]:
df['city'].value_counts()[df['city'].value_counts()>600].sum()

43149

* Out of 55046  only 43149 data left
* Extract the city name and convert into list

In [19]:
eligible_city = df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

In [20]:
eligible_city

['Colombo',
 'Mirpur',
 'Dubai',
 'Johannesburg',
 'London',
 'Pallekele',
 'Auckland',
 'Barbados',
 'Cape Town',
 'Sydney',
 'St Lucia',
 'Melbourne',
 'Lauderhill',
 'Wellington',
 'Durban',
 'Southampton',
 'Hamilton',
 'Centurion',
 'Mumbai',
 'Nottingham',
 'Kolkata',
 'Chittagong',
 'Delhi',
 'Abu Dhabi',
 'Mount Maunganui',
 'Manchester',
 'Chandigarh',
 'Lahore',
 'St Kitts',
 'Christchurch',
 'Trinidad',
 'Nagpur',
 'Cardiff']

In [21]:
df = df[df['city'].isin(eligible_city)]

In [22]:
df.shape

(43149, 7)

### current score
* Is ball ke kerne ke baad current score kya h 
* using the cummulative sum 

In [23]:
df['current_score'] = df.groupby('match_id').cumsum()['runs']

In [24]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6


### Ball left and wicket left
* ball contain over number with ball number --19.3  

In [25]:
df['over']= df['ball'].apply(lambda x: str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])

In [26]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5


In [27]:
df.dtypes

match_id              int64
batting_team         object
bowling_team         object
ball                float64
runs                  int64
player_dismissed     object
city                 object
current_score         int64
over                 object
ball_no              object
dtype: object

**Ball Bowled**

In [28]:
# ball bowled   over*6 + ball_no

df['balls_bowled'] = (df['over'].astype('int')*6) +(df['ball_no'].astype('int')) 

In [29]:
df.head(10)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5
5,2,Pakistan,Sri Lanka,0.6,1,0,Johannesburg,7,0,6,6
6,2,Pakistan,Sri Lanka,1.1,0,0,Johannesburg,7,1,1,7
7,2,Pakistan,Sri Lanka,1.2,1,0,Johannesburg,8,1,2,8
8,2,Pakistan,Sri Lanka,1.3,2,0,Johannesburg,10,1,3,9
9,2,Pakistan,Sri Lanka,1.4,0,0,Johannesburg,10,1,4,10


**Ball Left**

In [30]:
120- df['balls_bowled'].unique()


array([119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107,
       106, 105, 104, 103, 102, 101, 100,  99,  98,  97,  96,  95,  94,
        93,  92,  91,  90,  89,  88,  87,  86,  85,  84,  83,  82,  81,
        80,  79,  78,  77,  76,  75,  74,  73,  72,  71,  70,  69,  68,
        67,  66,  65,  64,  63,  62,  61,  60,  59,  58,  57,  56,  55,
        54,  53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,  42,
        41,  40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,  29,
        28,  27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,
        15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,
         2,   1,   0,  -1,  -2,  -3])

* In this some of the value are in negative number so  we neglate it .

In [31]:
df['ball_left'] = 120 - df['balls_bowled']
# for extra 
df['ball_left'] = df['ball_left'].apply(lambda x: 0 if x<0 else x)
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,ball_left
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1,119
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2,118
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3,117
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4,116
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5,115


In [32]:
df['ball_left'].unique()

array([119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107,
       106, 105, 104, 103, 102, 101, 100,  99,  98,  97,  96,  95,  94,
        93,  92,  91,  90,  89,  88,  87,  86,  85,  84,  83,  82,  81,
        80,  79,  78,  77,  76,  75,  74,  73,  72,  71,  70,  69,  68,
        67,  66,  65,  64,  63,  62,  61,  60,  59,  58,  57,  56,  55,
        54,  53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,  42,
        41,  40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,  29,
        28,  27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,
        15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,
         2,   1,   0])

* All are in positive format

**Wicket Left**

In [33]:
df['player_dismissed'].unique()

array(['0', 'Imran Nazir', 'Salman Butt', 'Mohammad Hafeez',
       'Younis Khan', 'Shoaib Malik', 'Shahid Afridi', 'AJ Finch',
       'CA Lynn', 'DJM Short', 'MP Stoinis', 'GJ Maxwell', 'AT Carey',
       'NM Coulter-Nile', 'Karim Sadiq', 'Mohammad Shahzad',
       'Nawroz Mangal', 'Noor Ali Zadran', 'Asghar Stanikzai',
       'Mohammad Nabi', 'Samiullah Shenwari', 'Hamid Hassan',
       'RR Hendricks', 'MN van Wyk', 'F du Plessis', 'DA Miller',
       'LD Chandimal', 'GSNFG Jayasuriya', 'CK Kapugedera', 'MD Shanaka',
       'HD Rutherford', 'MJ Guptill', 'LRPL Taylor', 'C Munro',
       'GD Elliott', 'BB McCullum', 'DA Warner', 'RT Ponting',
       'SR Watson', 'BJ Haddin', 'MJ Clarke', 'MEK Hussey', 'DJ Hussey',
       'B Lee', 'NM Hauritz', 'DR Smith', 'MC Bascombe', 'J Charles',
       'CD Barnwell', 'DJG Sammy', 'Babar Azam', 'Sahibzada Farhan',
       'Asif Ali', 'Sarfraz Ahmed', 'C de Grandhomme', 'KS Williamson',
       'Sharjeel Khan', 'Khurram Manzoor', 'Umar Akmal', 'Wahab 

* In case data is in format of character we want to convert into numeric so :-
* not out --> "0"------> 0
* else 1
* apply the cummulative sum and then counts the number of wicket 

In [37]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x == '0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed']= df.groupby('match_id').cumsum()['player_dismissed']
df['wicket_left'] = 10 - df['player_dismissed']

In [42]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,ball_left,wicket_left
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1,119,10
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2,118,10
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3,117,10
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4,116,10
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,823,India,Australia,19.2,4,6,Colombo,137,19,2,116,4,4
118,823,India,Australia,19.3,0,7,Colombo,137,19,3,117,3,3
119,823,India,Australia,19.4,1,7,Colombo,138,19,4,118,2,3
120,823,India,Australia,19.5,1,7,Colombo,139,19,5,119,1,3


**Calculater the current run rate**
* (run*6)/no.of ball 

In [43]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']

In [44]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,ball_left,wicket_left,crr
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1,119,10,0.000000
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2,118,10,12.000000
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3,117,10,10.000000
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4,116,10,9.000000
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5,115,10,7.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,823,India,Australia,19.2,4,6,Colombo,137,19,2,116,4,4,7.086207
118,823,India,Australia,19.3,0,7,Colombo,137,19,3,117,3,3,7.025641
119,823,India,Australia,19.4,1,7,Colombo,138,19,4,118,2,3,7.016949
120,823,India,Australia,19.5,1,7,Colombo,139,19,5,119,1,3,7.008403


**Rolling Function**
* In we can roll on the data frame in perticular window 
* eg :- last 10 rows , last 20 rows,.... etc
* In this case each match have initally 30 balls == null and after 30 ball it start show the sum of each 30 ball windows 
* **Extend Function**
* Extend Function is used to append the list and form single list

In [50]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window = 30).sum()['runs'].values.tolist())

In [53]:
df['last_five']= last_five

In [55]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,ball_left,wicket_left,crr,last_five
0,2,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1,119,10,0.0,NaN
1,2,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2,118,10,12.0,NaN
2,2,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3,117,10,10.0,NaN
3,2,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4,116,10,9.0,NaN
4,2,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5,115,10,7.2,NaN


**Final Score**

In [66]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df, on='match_id')
final_df.head()

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,current_score,over,ball_no,balls_bowled,ball_left,wicket_left,crr,last_five
0,2,189,Pakistan,Sri Lanka,0.1,0,0,Johannesburg,0,0,1,1,119,10,0.0,NaN
1,2,189,Pakistan,Sri Lanka,0.2,4,0,Johannesburg,4,0,2,2,118,10,12.0,NaN
2,2,189,Pakistan,Sri Lanka,0.3,1,0,Johannesburg,5,0,3,3,117,10,10.0,NaN
3,2,189,Pakistan,Sri Lanka,0.4,1,0,Johannesburg,6,0,4,4,116,10,9.0,NaN
4,2,189,Pakistan,Sri Lanka,0.5,0,0,Johannesburg,6,0,5,5,115,10,7.2,NaN


In [68]:
final_df = final_df[['batting_team','bowling_team','city','current_score','ball_left','wicket_left','crr','last_five','runs_x']]

In [69]:
final_df.head()

,batting_team,bowling_team,city,current_score,ball_left,wicket_left,crr,last_five,runs_x
0,Pakistan,Sri Lanka,Johannesburg,0,119,10,0.0,NaN,189
1,Pakistan,Sri Lanka,Johannesburg,4,118,10,12.0,NaN,189
2,Pakistan,Sri Lanka,Johannesburg,5,117,10,10.0,NaN,189
3,Pakistan,Sri Lanka,Johannesburg,6,116,10,9.0,NaN,189
4,Pakistan,Sri Lanka,Johannesburg,6,115,10,7.2,NaN,189


In [70]:
# drop the na value
final_df.dropna(inplace = True)

/home/arcgate/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
ball_left        0
wicket_left      0
crr              0
last_five        0
runs_x           0
dtype: int64

In [73]:
final_df

,batting_team,bowling_team,city,current_score,ball_left,wicket_left,crr,last_five,runs_x
29,Pakistan,Sri Lanka,Johannesburg,33,91,8,6.827586,33.0,189
30,Pakistan,Sri Lanka,Johannesburg,33,90,8,6.600000,33.0,189
31,Pakistan,Sri Lanka,Johannesburg,33,89,8,6.387097,29.0,189
32,Pakistan,Sri Lanka,Johannesburg,33,88,8,6.187500,28.0,189
33,Pakistan,Sri Lanka,Johannesburg,33,87,8,6.000000,27.0,189
...,...,...,...,...,...,...,...,...,...
43144,India,Australia,Colombo,137,4,4,7.086207,41.0,140
43145,India,Australia,Colombo,137,3,3,7.025641,36.0,140
43146,India,Australia,Colombo,138,2,3,7.016949,37.0,140
43147,India,Australia,Colombo,139,1,3,7.008403,38.0,140


### Note:-
**Kisi tera ka bias generate na ho isliye suffling kerege**

In [79]:
final_df = final_df.sample(final_df.shape[0])

In [80]:
final_df.head(2)

,batting_team,bowling_team,city,current_score,ball_left,wicket_left,crr,last_five,runs_x
12366,Pakistan,Australia,Dubai,50,82,9,7.894737,37.0,147
239,Australia,India,Melbourne,127,11,3,6.990826,52.0,132


**Train-Test split**

In [82]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state= 1)

In [83]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error

In [84]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse = False, drop='first'),['batting_team','bowling_team','city'])
]
, remainder = 'passthrough')

In [86]:
pipe = Pipeline(steps = [
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators = 1000, learning_rate = 0.2, max_depth = 12, random_state = 1))
])

In [88]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict((X_test))
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9860804238635877
1.6370217973496146


In [89]:
pickle.dump(pipe,open('pipe.pkl','wb'))